In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data_directory = 'data/'
image_directory = 'data/Pre_train/'

In [ ]:
ohe = OneHotEncoder()

In [ ]:
ohe.fit(np.arange(3).reshape(-1, 1))

In [ ]:
ohe.transform([[0], [1], [2]]).toarray()

In [ ]:
size = 32
batch_size = 256
num_channels = 3
training_iters = 50
display = 1
vali_batch_size =  500

In [ ]:
def read_label_files(csv_file):
    file = open(data_directory+csv_file, 'r')
    filepaths = []
    labels = []
    for i in file:
        filename, label = i.split(',')
        filepaths.append(image_directory+filename)
        labels.append(int(label))
    return filepaths, labels    

In [ ]:
train_path, train_labels = read_label_files('train_set.csv')
vali_path, vali_labels = read_label_files('vali_set.csv')

In [ ]:
vali_batch_size = len(vali_labels)

In [ ]:
train_path_tensor = ops.convert_to_tensor(train_path, dtype=dtypes.string)
vali_path_tensor = ops.convert_to_tensor(vali_path, dtype=dtypes.string)
train_labels_tensor = ops.convert_to_tensor(train_labels, dtype=dtypes.int32)
vali_labels_tensor = ops.convert_to_tensor(vali_labels, dtype=dtypes.int32)

In [ ]:
train_input_queue = tf.train.slice_input_producer(
                                    [train_path_tensor, train_labels_tensor],
                                    shuffle=True)
vali_input_queue = tf.train.slice_input_producer(
                                    [vali_path_tensor, vali_labels_tensor],
                                    shuffle=True)

In [ ]:
file_content = tf.read_file(train_input_queue[0])
train_image = tf.image.decode_jpeg(file_content, channels=num_channels)
train_image /= 255
train_label = train_input_queue[1]

file_content = tf.read_file(vali_input_queue[0])
vali_image = tf.image.decode_jpeg(file_content, channels=num_channels)
vali_image /= 255
vali_label = vali_input_queue[1]

In [ ]:
train_image.set_shape([size, size, num_channels])
vali_image.set_shape([size, size, num_channels])

train_batch = tf.train.batch(
                            [train_image, train_label],
                            batch_size= batch_size
                            )
vali_batch = tf.train.batch(
                            [vali_image, vali_label],
                            batch_size = vali_batch_size
                            )

In [ ]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)
    
#     for i in range(1):
#         test = sess.run(train_batch)
#     coord.request_stop()
#     coord.join(threads)

In [ ]:
# image, label = test

In [ ]:
# print(image.shape)
# print(label.shape)

In [ ]:
X = tf.placeholder(tf.float32, shape = [None, size, size, 3])
y = tf.placeholder(tf.float32, shape=[None, 3])

In [ ]:
weights = {
        'wc1':tf.Variable(tf.random_normal([5, 5, 3, 32])),
        'wf1':tf.Variable(tf.random_normal([16 * 16 * 32, 1024])),
        'out':tf.Variable(tf.random_normal([1024, 3]))
         }

In [ ]:
bias = {
        'bc1' : tf.Variable(tf.random_normal([32])),
        'bf1' : tf.Variable(tf.random_normal([1024])),
        'out':tf.Variable(tf.random_normal([3]))
        }

In [ ]:
def conv_net(input_image):
        conv1 = tf.nn.conv2d(input_image, weights['wc1'], [1, 1, 1, 1], padding='SAME')
        conv1 = tf.add(conv1, bias['bc1'])
        max1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        b, h, w, c = max1.get_shape().as_list()
        unrolled = tf.reshape(max1, [-1, h * w * c])
        full1 = tf.add(tf.matmul(unrolled, weights['wf1']), bias['bf1'])
        out = tf.add(tf.matmul(full1, weights['out']), bias['out'])
        return out

In [ ]:
pred = conv_net(X)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
train_op = tf.train.AdamOptimizer().minimize(loss)
corrected_pred = tf.equal(tf.arg_max(pred, 1), tf.arg_max(y, 1))
acc = tf.reduce_mean(tf.cast(corrected_pred, tf.float32))

In [ ]:
loss_list = []
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in range(training_iters):
        image, label = sess.run(train_batch)
        label = ohe.transform(label.reshape(-1, 1)).toarray()
        temp, _ = sess.run([loss, train_op], feed_dict={X:image, y:label})
        if i % display == 0:
            val_img, val_lab = sess.run(vali_batch)
            val_lab = ohe.transform(val_lab.reshape(-1, 1)).toarray()
            val_loss, accuracy = sess.run([loss, acc], feed_dict={X:val_img, y:val_lab})
            print('Iter %s, Validation Acc %.4f, Training loss %.4f'%(i, accuracy, val_loss))
        
    coord.request_stop()
    coord.join(threads)    

In [ ]:
loss_list